# Analysis of Customers' Usage of Cyclistic Bikes using R

This is a notebook by Chialuka Emeana

## Introduction

In 2016, Cyclistic launched a successful bike-share offering. Since then, the offering has grown to a fleet of 5,824 bicycles that are geotracked and locked into a network of 692 stations across chicago. Until now, Cyclistic's marketing strategy relied on building general awareness and appealing to broad consumer segments. The director of marketing has a set goal of designing marketing strategies aimed at converting casual riders into annual members.

As a junior data analyst in the marketing analyst team, I have been tasked to help the team understand how casual riders and annual members use cyclistic bikes differently by backing up my recommendations with compelling data insights and Data Visualization techniques. I performed my analysis based on the following criteria:

• Total number of rides

• Average trip duration

## Contents
1. Importing Libraries
2. Importing Datasets
3. Data Wrangling
4. Stack into a Single Data Frame
5. Cleaning up and Adding Data to Prepare for Analysis
6. Conducting Descriptive Analysis
7. Visualizations
8. Exporting Summary Files for Further Analysis

## 1. Importing Libraries
Notes: setting up my environment by loading the required packages
#### 'tidyverse' for data import and wrangling
#### 'lubridate' for date functions
#### 'ggplot2' for visualization
#### 'skimr' for compact data summary

In [ ]:
library(tidyverse)
library(lubridate)
library(ggplot2)
library(skimr)

## 2. Importing Datasets
Notes: uploading the 12 months of data for the analysis using the 'read_csv' function and assigning them to individual data frames.
Data for this analysis was made available by Motivate International Inc. and can be found [here](http://divvy-tripdata.s3.amazonaws.com/index.html) and licensed under this [agreement](http://ride.divvybikes.com/data-license-agreement)


In [ ]:
m6_2021 <- read_csv("../input/cyclistic-bike-share-google-capstone/202106-divvy-tripdata.csv")
m7_2021 <- read_csv("../input/cyclistic-bike-share-google-capstone/202107-divvy-tripdata.csv")
m8_2021 <- read_csv("../input/cyclistic-bike-share-google-capstone/202108-divvy-tripdata.csv")
m9_2021 <- read_csv("../input/cyclistic-bike-share-google-capstone/202109-divvy-tripdata.csv")
m10_2021 <- read_csv("../input/cyclistic-bike-share-google-capstone/202110-divvy-tripdata.csv")
m11_2021 <- read_csv("../input/cyclistic-bike-share-google-capstone/202111-divvy-tripdata.csv")
m12_2021 <- read_csv("../input/cyclistic-bike-share-google-capstone/202112-divvy-tripdata.csv")
m1_2022 <- read_csv("../input/cyclistic-bike-share-google-capstone/202201-divvy-tripdata.csv")
m2_2022 <- read_csv("../input/cyclistic-bike-share-google-capstone/202202-divvy-tripdata.csv")
m3_2022 <- read_csv("../input/cyclistic-bike-share-google-capstone/202203-divvy-tripdata.csv")
m4_2022 <- read_csv("../input/cyclistic-bike-share-google-capstone/202204-divvy-tripdata.csv")
m5_2022 <- read_csv("../input/cyclistic-bike-share-google-capstone/202205-divvy-tripdata.csv")

## 3. Data Wrangling


### - Comparing column names

In [ ]:
# I compare the column names to ensure they are in the same order for all tables.
colnames(m6_2021)
colnames(m7_2021)
colnames(m8_2021)
colnames(m9_2021)
colnames(m10_2021)
colnames(m11_2021)
colnames(m12_2021)
colnames(m1_2022)
colnames(m2_2022)
colnames(m3_2022)
colnames(m4_2022)
colnames(m5_2022)
# Column names are consistent
# so I proceed to the next step which is checking for inconsistency in data format.

### - Inspect for incongruencies

In [ ]:
# I check the data formats using the 'str()' function for any irregularities in the data structures.
str(m6_2021)
str(m7_2021)
str(m8_2021)
str(m9_2021)
str(m10_2021)
str(m11_2021)
str(m12_2021)
str(m1_2022)
str(m2_2022)
str(m3_2022)
str(m4_2022)
str(m5_2022)

#### By inspecting for incongruencies, I found the 'started_at' and 'ended_at' columns in 'm5_2022' data frame have their variables in character formats rather than date formats. 

### - Converting both columns in 'm5_2022' to date format so they can stack correctly.

In [ ]:
m5_2022 <- mutate(m5_2022,
                  started_at = as.POSIXct(started_at,
                                          format = "%m/%d/%Y %H:%M",
                                          tz=Sys.timezone()),
                  ended_at = as.POSIXct(ended_at,
                                        format = "%m/%d/%Y %H:%M",
                                        tz=Sys.timezone()))

In [ ]:
# Checking again to confirm that the format is correct.
str(m5_2022)

## 4. Stack into one Data Frame
Notes: Since all the columns in the monthly data frames are consistent, I combine them using the "bind_rows" function.

In [ ]:
all_trips <-bind_rows(m6_2021, m7_2021, m8_2021, m9_2021, m10_2021, m11_2021,
                      m12_2021, m1_2022, m2_2022, m3_2022, m4_2022, m5_2022)

### - Dropping the columns not needed.
Next, I decide to drop the latitude and longitude columns as I would not be
needing them for this analysis.


In [ ]:
all_trips <- all_trips %>%  
  select(-c(start_lat, start_lng, end_lat, end_lng))

Now, the table has 9 columns in total.

## 5. Cleaning up and Adding Data to Prepare for Analysis

### - Inspecting the new table

In [ ]:
# Checking the data frame using the 'colnames','nrows', 'dim', 'head', 'tail', 'str' and 'summary' functions.
colnames(all_trips)

nrow(all_trips)

dim(all_trips)

head(all_trips)

tail(all_trips)

str(all_trips)

summary(all_trips)

### - Checking to see how many observations fall under each user type.

In [ ]:
table(all_trips$member_casual)

#### This confirms that there are 3156750 member and 2428895 casual observations.

### - Adding columns that list the date, month, day, year and time of each ride.

In [ ]:
all_trips$date <- as.Date(all_trips$started_at) #The default format is yyyy-mm-dd
all_trips$month <- format(as.Date(all_trips$date), "%m")
all_trips$day <- format(as.Date(all_trips$date), "%d")
all_trips$year <- format(as.Date(all_trips$date), "%Y")
all_trips$day_of_week <- format(as.Date(all_trips$date), "%A")
all_trips$time <- format(all_trips$started_at,"%H: %M: %S")
all_trips$time <- hms(all_trips$time)

### - Adding a "trip_duration" calculation to all_trips (in minutes)

In [ ]:
all_trips$trip_duration <- round(difftime(all_trips$ended_at,
                                          all_trips$started_at, units = "mins"))

### - Inspecting the structures of the columns

In [ ]:
str(all_trips)

### - Converting "trip_duration" from factor to numeric

#### This is so I can run calculations on the data

In [ ]:
all_trips$trip_duration <- as.numeric(as.character(all_trips$trip_duration))
is.numeric(all_trips$trip_duration)

### - Adding a "time_of_day" column
Notes: Morning represents the period from 6am to 11am, Afternoon represents the period from 12 noon to 3pm, Evening represents the period from 4pm to 8pm and Night represents the period from 9pm to 5am.

In [ ]:
all_trips <- all_trips %>%
  mutate(time_of_day = case_when(time >= "6H 00M 00S" & time < "12H 00M 00S" ~ 'morning',
                              time >= "12H 00M 00S" & time < "16H 00M 00S" ~ "afternoon",
                              time >= "16H 00M 00S" & time < "21H 00M 00S" ~ "evening", TRUE ~ "night"))


### - Removing bad data
Notes: The table contains a few entries when bikes were taken out of docks and checked for quality by Cyclistic and negative trip_duration. 
I create a new version of the data frame(v2) since data is being removed.

In [ ]:
all_trips_v2 <- 
  all_trips[!(all_trips$start_station_name == "HQ QR" | all_trips$trip_duration<0),]

### - Checking the head of the new data frame

In [ ]:
head(all_trips_v2)

### - Removing columns not needed for analysis
I am removing the rider_id, start_station_id, end_station_name, end_station_id
and time columns as I do not need the columns for my analysis.
I create a new version of the data frame (v3) since data is being removed.

In [ ]:
all_trips_v3 <- all_trips_v2 %>%
  select(-c(ride_id, start_station_id, end_station_name, end_station_id, time))

### - Removing NA values
Notes: The table now contains "NA" values which I would like to omit to reduce the number of rows for my analysis. Thus, I create a new version of the data frame(v4) since data is being removed.

In [ ]:
all_trips_v4 <- na.omit(all_trips_v3)
head(all_trips_v4)

## 6. Conducting Descriptive Analysis

### - Descriptive analysis on trip_duration (figures in minutes)
The average of all trip duration is 21.51 minutes. The midpoint of all trip duration is 12 minutes. The shortest ride is 0 minutes and the longest ride is 
55,944 minutes.

In [ ]:
summary(all_trips_v4$trip_duration)

In [ ]:
table(all_trips_v4$member_casual)

### - Comparing members and casual users

#### (1) By average trip duration

In [ ]:
aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual, FUN = mean)

#### Casual users have an average duration of 32 mins which is higher than that of members which is 13 mins.

#### (2) By median trip duration

In [ ]:
aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual, FUN = median)

#### Notes: Casual users have a midpoint duration of 16 minutes which is higher than that of members which is 9 minutes.



#### (3) By maximum trip duration

In [ ]:
aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual, FUN = max)

#### Casual users have a maximum duration of 55944 minutes mins which is higher than that of members which is 1560 minutes.


#### (4) By minimum trip duration

In [ ]:
aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual, FUN = min)

#### Casual users and members have a minimum trip duration of zero minutes.

### - Checking the average trip duration by month for each user

In [ ]:
aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual +
            all_trips_v4$month, FUN = mean)

### - Checking the average trip duration by day of week

In [ ]:
aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual +
            all_trips_v4$day_of_week, FUN = mean)

### - Fixing the order of the day of the week

In [ ]:
# Seeing as the days of the week are out of order, I would reorder it and run the code again.
all_trips_v4$day_of_week <- ordered(all_trips_v4$day_of_week,
                                    levels=c("Sunday", "Monday", "Tuesday",
                                             "Wednesday", "Thursday", "Friday",
                                             "Saturday"))


aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual +
            all_trips_v4$day_of_week, FUN = mean)


### - Checking the average trip duration by time of day

In [ ]:
aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual +
            all_trips_v4$time_of_day, FUN = mean)

### - Fixing the order of the time of day

In [ ]:
# Seeing as the time of the day are out of order, I would reorder it.
all_trips_v4$time_of_day <- ordered(all_trips_v4$time_of_day,
                                    levels=c("morning", "afternoon", "evening",
                                             "night"))

# Running the code again
aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual +
            all_trips_v4$time_of_day, FUN = mean)

### - Checking the number of rides by bike type

In [ ]:
all_trips_v4 %>%
  group_by(rideable_type, member_casual) %>%
  summarize(num_of_rides = n(), .groups = 'drop')

### - Checking the number of rides by month for members vs casual users

In [ ]:
num_of_rides_by_month <- all_trips_v4 %>%
  group_by(month, member_casual) %>%
  summarize(num_of_rides = n(), .groups = 'drop')


head(num_of_rides_by_month)

### - Checking to see top start stations used by members

In [ ]:
member_start_stations <- all_trips_v4 %>%
  filter(member_casual == "member" & start_station_name != "na") %>%
  group_by(start_station_name, member_casual) %>%
  summarize(num_of_rides = n(), .groups = 'drop') %>%
  arrange(-num_of_rides)

head(member_start_stations)

### - Checking to see top start stations used by casual users

In [ ]:
casual_start_stations <- all_trips_v4 %>%
  filter(member_casual == "casual" & start_station_name != "na") %>%
  group_by(start_station_name, member_casual) %>%
  summarize(num_of_rides = n(), .groups = 'drop') %>%
  arrange(-num_of_rides)

head(casual_start_stations)

## 7. Visualizations

### - The number of rides by user

In [ ]:
all_trips_v4 %>%
  group_by(member_casual, day_of_week) %>%
  summarise(number_of_rides=n(), average_duration = mean(trip_duration), .groups = 'drop') %>%
  arrange(member_casual, day_of_week) %>%
  ggplot(aes(x=day_of_week, y=number_of_rides, fill = member_casual)) +
  geom_col(position = "dodge")

#### Notes: Saturday has the highest number of rides for casual users and Tuesday was the day members recorded the highest number of rides.

### - Daily average duration by user

In [ ]:
all_trips_v4 %>%
  group_by(member_casual, day_of_week) %>%
  summarise(number_of_rides=n(), average_duration = mean(trip_duration), .groups = 'drop') %>%
  arrange(member_casual, day_of_week) %>%
  ggplot(aes(x=day_of_week, y=average_duration, fill = member_casual)) +
  geom_col(position = "dodge")

#### Notes: The highest average duration was recorded on Sundays for both members and casual users.

## 8. Exporting Summary Files for Further Analysis

This is the last step of my analyze phase and here, I am creating csv files that I will visualize in Tableau.

#### (1) Average trip duration by day of week

In [ ]:
counts_1 <- aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual +
                        all_trips_v4$day_of_week, FUN = mean)
write.csv(counts_1, 
          file = './avg_trip_duration_by_day.csv')

#### (2) Average trip duration by time of day

In [ ]:
counts_2 <- aggregate(all_trips_v4$trip_duration ~ all_trips_v4$member_casual +
                        all_trips_v4$time_of_day, FUN = mean)
write.csv(counts_2, 
          file = './avg_trip_duration_by_time_of_day.csv')


#### (3) Number of rides by bike type

In [ ]:
counts_3  <- all_trips_v4 %>%
  group_by(rideable_type, member_casual) %>%
  summarize(num_of_rides = n(), .groups = 'drop')

write.csv(counts_3, 
          file = './number_of_rides_by_bike_type.csv')

#### (4) Number of rides by month

In [ ]:
counts_4 <- all_trips_v4 %>%
  group_by(month, member_casual) %>%
  summarize(num_of_rides = n(), .groups = 'drop')

write.csv(counts_4, 
          file = './number_of_user_rides_per_month.csv')


## View Tableau dashboard [here](http://ride.divvybikes.com/data-license-agreement) 
## Thanks for viewing this Notebook. 